#### Step 1 : Define the schema for the constructors data

In [0]:
%run "../includes/configuration"

In [0]:
constructors_schame = "constructorId INT, constructorRef STRING, name STRING, nationality STRING, url STRING"

In [0]:
constructors_df = spark.read   \
    .schema(constructors_schame)  \
    .json(f"{raw_folder_path}/constructors.json")

#### Step 2 : Drop the column url from the dataFrame

In [0]:
constructors_dropped_df = constructors_df.drop("url")

#### Step 3 : rename the 2 columns and add a new column as Ingestion date


In [0]:
from pyspark.sql.functions import col, current_timestamp, lit

In [0]:
v_source_file = dbutils.widgets.get("p_source_file")

In [0]:
constructors_final_df = constructors_dropped_df.withColumnRenamed("constructorRef", "constructor_ref")  \
                                               .withColumnRenamed("constructorId","constructor_id") \
                                               .withColumn("data_source", lit(v_source_file))

In [0]:
%run "../functions/common_functions"

In [0]:
constructors_final_df = add_ingest_date(constructors_final_df)

#### Step 4 : Save the data to parquet file with overwrite mode.

In [0]:
constructors_final_df.write.mode("overwrite").parquet(f"{processed_folder_path}/constructors")

In [0]:
display(spark.read.parquet(f"{processed_folder_path}/constructors"))

In [0]:
dbutils.notebook.exit("success")